In [1]:
!pip install ultralytics torch opencv-python scikit-learn pyyaml xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 3.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


GPU Available: False
GPU Name: No GPU detected


In [3]:
import os
import yaml
import torch
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import os

def load_data(root_dir):
    """
    Load data from the specified root directory
    
    Args:
        root_dir (str): Root directory containing subject folders
    
    Returns:
        tuple: Lists of image paths and corresponding labels
    """
    data = []
    labels = []
    for subject in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject)
        if os.path.isdir(subject_path):  # Check if it's a directory
            for activity_type in os.listdir(subject_path):
                activity_path = os.path.join(subject_path, activity_type)
                if os.path.isdir(activity_path):  # Check if it's a directory
                    label = 1 if activity_type == 'fall' else 0
                    for activity_class in os.listdir(activity_path):
                        class_path = os.path.join(activity_path, activity_class)
                        if os.path.isdir(class_path):  # Check if it's a directory
                            for file in os.listdir(class_path):
                                file_path = os.path.join(class_path, file)
                                if os.path.isfile(file_path):  # Ensure it's a file
                                    data.append(file_path)
                                    labels.append(label)
    return data, labels


In [5]:
train_data, train_labels = load_data("/kaggle/input/dataslayerdatasets/train")
print(f"Total images loaded: {len(train_data)}")

Total images loaded: 4294


In [6]:
import csv
import os
import shutil
from ultralytics import YOLO
import tensorflow as tf

# Create a directory for YOLO predictions
yolo_input_dir = "/kaggle/working/yolo_input"
os.makedirs(yolo_input_dir, exist_ok=True)

# Copy images into YOLO input directory with unique filenames and validation
for img_path in train_data:
    if os.path.exists(img_path) and os.path.isfile(img_path):
        # Create a unique filename to prevent overwriting
        unique_filename = "_".join(img_path.split(os.sep)[-3:])  # Use parent folders + filename
        unique_filepath = os.path.join(yolo_input_dir, unique_filename)
        shutil.copy(img_path, unique_filepath)
    else:
        print(f"File not found or inaccessible: {img_path}")

# Verify the total number of files copied
copied_files = os.listdir(yolo_input_dir)
print(f"Total files copied to YOLO input directory: {len(copied_files)}")



# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    
    
    # Initialize YOLOv8n-pose model
    model = YOLO('yolov8n-pose.pt')
    
    # Run predictions on the copied images
    results = model.predict(
        source=yolo_input_dir,  # Directory containing images
        task="pose",
        imgsz=640,
        verbose=False,
        save=True  # Save predictions
    )
    
    # Output directory for saved results
    print(f"Predictions saved in: {results[0].save_dir}")
    
    # train model normally


E0000 00:00:1734356178.291897      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1216 13:36:18.301716056      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1216 13:36:18.301735086      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1216 13:36:18.301738914      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1216 13:36:18.301741424      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1216 13:36:18.301743871      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

Total files copied to YOLO input directory: 4294
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1734356207.948698      13 service.cc:145] XLA service 0x5a53d5e0fdb0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734356207.948818      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1734356207.948827      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1734356207.948834      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1734356207.948840      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1734356207.948847      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1734356207.948854      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1734356207.948860      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1734356207.948867      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

100%|██████████| 6.52M/6.52M [00:00<00:00, 181MB/s]



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/pose/predict
Predictions saved in: runs/pose/predict


In [7]:
import csv
import os
import numpy as np

# Define the output CSV path
output_csv = "/kaggle/working/yolo_keypoints.csv"

# Open the CSV file for writing
with open(output_csv, mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the CSV header
    # Header: image_name, keypoint_1_x, keypoint_1_y, ..., keypoint_17_x, keypoint_17_y
    header = ['image_name']
    for i in range(1, 18):  # Assuming 17 keypoints for pose
        header.extend([f'keypoint_{i}_x', f'keypoint_{i}_y'])
    writer.writerow(header)

            # Iterate over the results and extract keypoints
    for result in results:
        image_name = os.path.basename(result.path)  # Get the image filename
    
        if hasattr(result, 'keypoints') and result.keypoints is not None and len(result.keypoints) > 0:
            # Access the first detected pose (unwrap the first dimension)
            keypoints = result.keypoints[0].cpu().numpy()  # Get raw keypoint data
            
            # Convert to numpy array with only (x, y) coordinates
            keypoints_xy = result.keypoints.xy[0].cpu().numpy()  # (17, 2)
    
            # Verify the keypoint shape
            if keypoints_xy.shape == (17, 2):
                # Prepare keypoint data for CSV
                keypoint_data = [image_name]  # Start with the image name
                
                # Extract x, y coordinates for each keypoint
                for kp in keypoints_xy:
                    keypoint_data.extend([kp[0], kp[1]])  # Only x, y (ignore confidence)
    
                # Write keypoint data to the CSV file
                writer.writerow(keypoint_data)
            else:
                print(f"Unexpected keypoint shape for image {image_name}: {keypoints_xy.shape}")
        else:
            # Log the issue if no keypoints are detected
            print(f"No keypoints detected for image: {image_name}")
    print(f"Keypoint data saved to: {output_csv}")


Unexpected keypoint shape for image fall_2_forward_falls_frame018.jpg: (0, 2)
Unexpected keypoint shape for image fall_2_right_falls_frame021.jpg: (0, 2)
Unexpected keypoint shape for image fall_2_right_falls_frame022.jpg: (0, 2)
Unexpected keypoint shape for image fall_3_standing_falls_frame012.jpg: (0, 2)
Unexpected keypoint shape for image fall_4_left_falls_frame042.jpg: (0, 2)
Unexpected keypoint shape for image fall_4_left_falls_frame043.jpg: (0, 2)
Unexpected keypoint shape for image fall_4_left_falls_frame045.jpg: (0, 2)
Keypoint data saved to: /kaggle/working/yolo_keypoints.csv


In [8]:
import pandas as pd

# Load the CSV file
data = pd.read_csv("/kaggle/input/keypoints/yolo_keypoints (1).csv")

# Automatically label the data based on the image_name
def label_image(image_name):
    image_name = image_name.lower()  # Convert to lowercase for case-insensitive comparison
    if "non_fall" in image_name:  # Check "non_fall" first to avoid it being overridden by "fall"
        return 0  # Label as non-fall
    elif "fall" in image_name:  # Check "fall" afterward
        return 1  # Label as fall
    else:
        return None 

# Apply the function to create a 'label' column
data['label'] = data['image_name'].apply(label_image)

# Drop rows with missing labels (if any image_name doesn't contain 'fall' or 'non_fall')
data = data.dropna(subset=['label'])

# Save the updated CSV (optional)
output_csv = "/kaggle/working/yolo_keypoints_labeled.csv"
data.to_csv(output_csv, index=False)

print(f"Labeled data saved to: {output_csv}")


Labeled data saved to: /kaggle/working/yolo_keypoints_labeled.csv


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import joblib

# Step 1: Load the labeled data
data = pd.read_csv("/kaggle/working/yolo_keypoints_labeled.csv")

# Step 2: Split into features (X) and labels (y)
# Drop 'image_name' since it's not a feature for training
X = data.drop(columns=['image_name', 'label'])
y = data['label']

# Step 3: Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 4: Train the XGBoost Classifier
model = xgb.XGBClassifier(
    n_estimators=300,   # Number of trees
    learning_rate=0.1,  # Step size shrinkage
    max_depth=5,        # Maximum depth of trees
    random_state=42,    # Seed for reproducibility
    use_label_encoder=False
)

# Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Step 6: Save the Model
output_model = "/kaggle/working/xgboost_fall_detection_model.pkl"
joblib.dump(model, output_model)
print(f"Model saved to: {output_model}")


/usr/local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [13:40:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.99
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       550
         1.0       1.00      0.98      0.99       308

    accuracy                           0.99       858
   macro avg       0.99      0.99      0.99       858
weighted avg       0.99      0.99      0.99       858

Model saved to: /kaggle/working/xgboost_fall_detection_model.pkl


In [10]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import shutil
from ultralytics import YOLO

def load_test_images_in_batches(test_dir, target_size=(640, 640), batch_size=32):
    """
    Loads and preprocesses images from the test folder in batches.
    """
    image_files = [f for f in os.listdir(test_dir) if f.endswith(('png', 'jpg', 'jpeg'))]

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        batch_images = []
        batch_ids = []

        for file in batch_files:
            img_path = os.path.join(test_dir, file)
            try:
                # Load and preprocess the image
                img = load_img(img_path, target_size=target_size)
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                batch_images.append(img_array)
                batch_ids.append(file)
            except Exception as e:
                print(f"Error processing {file}: {e}")

        yield np.array(batch_images), batch_ids

# Step 1: Test Directory
test_dir = '/kaggle/input/dataslayerdatasets/test'

# Step 2: Validate File Paths
if not os.path.exists(test_dir) or len(os.listdir(test_dir)) == 0:
    raise FileNotFoundError(f"No images found in directory: {test_dir}")

# Step 3: Create YOLO Input Directory
yolo_input_dir = "/kaggle/working/test"
os.makedirs(yolo_input_dir, exist_ok=True)

# Copy images into YOLO input directory with unique filenames and validation
image_files = [f for f in os.listdir(test_dir) if f.endswith(('png', 'jpg', 'jpeg'))]
for img_file in image_files:
    img_path = os.path.join(test_dir, img_file)
    if os.path.exists(img_path):
        unique_filepath = os.path.join(yolo_input_dir, img_file)  # Avoid modifying filenames
        shutil.copy(img_path, unique_filepath)
    else:
        print(f"File not found or inaccessible: {img_path}")

# Verify the total number of files copied
copied_files = os.listdir(yolo_input_dir)
print(f"Total files copied to YOLO input directory: {len(copied_files)}")

# Step 4: Initialize YOLOv8 Pose Model
model = YOLO('yolov8n-pose.pt')

# Step 5: Run YOLO Predictions
results = model.predict(
    source=yolo_input_dir,  # Directory containing images
    task="pose",
    imgsz=640,
    verbose=False,
    save=True  # Save predictions
)

# Output the saved results directory
print(f"Predictions saved in: {results[0].save_dir}")



Total files copied to YOLO input directory: 2152

WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/pose/predict2
Predictions saved in: runs/pose/predict2


In [11]:
import csv
import os

# Save keypoints to CSV
output_csv = "/kaggle/working/yolo_predictions.csv"
with open(output_csv, mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header: image_name + flattened keypoints (x, y for 17 keypoints)
    header = ['image_name'] + [f'keypoint_{i}_x' for i in range(1, 18)] + [f'keypoint_{i}_y' for i in range(1, 18)]
    writer.writerow(header)

    # Iterate through results
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None:
            keypoints_tensor = result.keypoints.xy.cpu().numpy()  # Extract keypoints (x, y)
            image_name = os.path.basename(result.path)  # Get the image filename

            for detection_keypoints in keypoints_tensor:  # Iterate over detections
                flattened_keypoints = detection_keypoints.flatten()  # Flatten (x, y) coordinates for 17 keypoints
                writer.writerow([image_name] + list(flattened_keypoints))  # Write to CSV

print(f"Keypoints saved to: {output_csv}")


Keypoints saved to: /kaggle/working/yolo_predictions.csv


In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

# Load the test set keypoints CSV
test_csv = "/kaggle/working/yolo_predictions.csv"
test_keypoints = pd.read_csv(test_csv)

# Extract image names and keypoints
image_names = test_keypoints['image_name']
keypoints = test_keypoints.drop('image_name', axis=1).values

# Preprocess keypoints (skip scaling if scaler is unavailable)
# Replace this with the appropriate saved scaler loading code if available
try:
    from joblib import load
    scaler = load("/kaggle/working/scaler.pkl")
    keypoints_scaled = scaler.transform(keypoints)
except FileNotFoundError:
    print("Scaler not found. Proceeding without scaling.")
    keypoints_scaled = keypoints

# Load the pre-trained XGBoost model (saved using joblib)
from joblib import load
xgb_model = load("/kaggle/working/xgboost_fall_detection_model.pkl")

# Make predictions
predictions = xgb_model.predict(keypoints_scaled)

# Convert probabilities to binary labels (assuming binary classification)
labels = (predictions > 0.5).astype(int)

# Save predictions with image names to a new CSV
output_csv = "/kaggle/working/test_predictions.csv"
output_df = pd.DataFrame({'id': image_names, 'label': labels})
output_df.to_csv(output_csv, index=False)

print(f"Predictions saved to: {output_csv}")


Scaler not found. Proceeding without scaling.
Predictions saved to: /kaggle/working/test_predictions.csv


In [13]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def load_test_images_in_batches(test_dir, target_size=(640, 640), batch_size=32):
    """
    Loads and preprocesses images from the test folder in batches.

    Args:
        test_dir (str): Directory containing test images.
        target_size (tuple): Desired image size (width, height).
        batch_size (int): Number of images to process per batch.

    Yields:
        tuple: Batch of images as a NumPy array and corresponding image IDs.
    """
    image_files = [f for f in os.listdir(test_dir) if f.endswith(('png', 'jpg', 'jpeg'))]

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        batch_images = []
        batch_ids = []

        for file in batch_files:
            img_path = os.path.join(test_dir, file)
            try:
                # Load and preprocess the image
                img = load_img(img_path, target_size=target_size)
                img_array = img_to_array(img) / 255.0  # Novrmalize to [0, 1]
                batch_images.append(img_array)
                batch_ids.append(file)
            except Exception as e:
                print(f"Error processing {file}: {e}")

        yield np.array(batch_images), batch_ids

# Example usage
test_dir = '/kaggle/input/dataslayerdatasets/test'
for batch_images, batch_ids in load_test_images_in_batches(test_dir):
    print(f"Processed batch of {len(batch_images)} images.")


Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed batch of 32 images.
Processed 

In [14]:
import csv
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# Function to load CSV data
def load_csv(file_path):
    """Loads a CSV file and returns a dictionary with photo ID as key and label as value."""
    data_dict = {}
    with open(file_path, mode='r') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        next(csv_reader)  # Skip the header
        
        for row in csv_reader:
            # Check if the row contains a valid format
            if len(row) != 2:
                print(f"Skipping invalid row: {row}")
                continue
            
            full_path_or_id, label = row
            # Extract photo ID from the full path if necessary
            photo_id = Path(full_path_or_id).name  # This ensures we only get the filename
            data_dict[photo_id] = int(label)  # Convert label to integer
    
    return data_dict

# Function to calculate accuracy and log incorrect predictions
def calculate_accuracy_and_log_incorrect(predicted_csv, ground_truth_csv):
    """
    Compares the predicted labels to the ground truth labels using the photo IDs as keys.
    Logs incorrect predictions for visualization.
    """
    # Load both CSVs into dictionaries
    predicted = load_csv(predicted_csv)
    ground_truth = load_csv(ground_truth_csv)
    
    # Initialize counters
    correct = 0
    total = 0
    incorrect_predictions = []  # Store tuples of (photo_id, predicted_label, true_label)
    
    for photo_id, true_label in ground_truth.items():
        # Check if the photo ID exists in the predicted data
        if photo_id in predicted:
            total += 1  # Increment total for every matched ID
            if predicted[photo_id] == true_label:
                correct += 1  # Increment correct if labels match
            else:
                incorrect_predictions.append((photo_id, predicted[photo_id], true_label))
        else:
            print(f"Warning: Photo ID {photo_id} not found in predictions.")
    
    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0
    return accuracy, correct, total, incorrect_predictions

import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import math

def visualize_incorrect_predictions(incorrect_predictions, image_dir, grid_size=5, resize_to=(100, 100)):
    """
    Displays the images for the incorrect predictions in a compact grid layout with resized images.
    """
    num_images = min(len(incorrect_predictions), grid_size * (grid_size *4))  # Max images to show
    rows = math.ceil(num_images / grid_size)
    cols = grid_size

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))  # Smaller figure size
    axes = axes.flatten()  # Flatten for easy iteration

    for idx, (photo_id, predicted_label, true_label) in enumerate(incorrect_predictions[:num_images]):
        image_path = Path(image_dir) / photo_id  # Construct the full image path
        
        # Check if the image exists
        if not image_path.exists():
            print(f"Image not found: {image_path}")
            axes[idx].axis("off")
            continue
        
        try:
            # Load and resize the image
            img = Image.open(image_path)
            img = img.resize(resize_to, Image.Resampling.LANCZOS)  # Resize to reduce load time
            
            # Display the image
            axes[idx].imshow(img)
            axes[idx].axis("off")
            axes[idx].set_title(f"Pred: {predicted_label}\nTrue: {true_label}", fontsize=8)
        except Exception as e:
            print(f"Error loading image {photo_id}: {e}")
            axes[idx].axis("off")
    
    # Hide any unused axes
    for idx in range(num_images, len(axes)):
        axes[idx].axis("off")
    
    plt.tight_layout(pad=1.0)
    plt.show()
# File paths for comparison
predicted_csv = '/kaggle/working/predictions_threshold_0.25.csv'  # Replace with your predicted CSV path
ground_truth_csv = '/kaggle/input/label-diri/file_labels (3).csv'  # Replace with your ground truth CSV path
image_directory = '/kaggle/input/dataslayerdatasets/test'  # Replace with the directory containing the images

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

def plot_confusion_matrix(y_true, y_pred, labels, title="Confusion Matrix"):
    """
    Generate and plot a confusion matrix with a heatmap.
    """
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()





In [16]:
# Example usage
predicted_csv = output_csv  # Replace with your predicted CSV path
ground_truth_csv = '/kaggle/input/label-diri/file_labels (3).csv'  # Replace with your ground truth CSV path

# Get accuracy and incorrect predictions
accuracy, correct, total, incorrect_predictions, y_true, y_pred = calculate_accuracy_and_log_incorrect(predicted_csv, ground_truth_csv)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}% ({correct}/{total} correct predictions)")

# Plot the confusion matrix
labels = [0, 1]  # Change if you have more classes
plot_confusion_matrix(y_true, y_pred, labels)


ValueError: not enough values to unpack (expected 6, got 4)

In [ ]:
# Example usage
predicted_csv = '/kaggle/working/test_predictions.csv'  # Replace with your predicted CSV path
ground_truth_csv = '/kaggle/input/label-diri/file_labels (3).csv'  # Replace with your ground truth CSV path

# Get accuracy and incorrect predictions
accuracy, correct, total, incorrect_predictions, y_true, y_pred = calculate_accuracy_and_log_incorrect(predicted_csv, ground_truth_csv)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}% ({correct}/{total} correct predictions)")

# Plot the confusion matrix
labels = [0, 1]  # Change if you have more classes
plot_confusion_matrix(y_true, y_pred, labels)
